In [18]:
import h5py
from sklearn.model_selection import train_test_split

In [2]:
data = h5py.File('data/Q1/MNIST_Subset.h5','r+')
X=data['X'][:]
Y=data['Y'][:]

In [7]:
X.shape,Y.shape

((14251, 28, 28), (14251,))

In [15]:
X = X.reshape(X.shape[0],-1)

In [19]:
?train_test_split

In [ ]:
X_train,X_test,Y_train,Y_test = te